In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import math
import matplotlib.pyplot as plt
from mpl_finance import candlestick2_ohlc
import pandas as pd
import numpy as np

In [5]:
import torch
from torch import nn
from PIL import Image

In [6]:
root_dir = '/notebooks/stock-price-prediction/'

In [7]:
df=pd.read_csv(root_dir + 'data/dji_2009-2019.csv')
print('Number of rows and columns:', df.shape)
length = df.shape[0]
df.head()

Number of rows and columns: (2767, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-01-02,8772.250000,9065.280273,8760.780273,9034.690430,9034.690430,213700000
1,2009-01-05,9027.129883,9034.370117,8892.360352,8952.889648,8952.889648,233760000
2,2009-01-06,8954.570313,9088.059570,8940.950195,9015.099609,9015.099609,215410000
3,2009-01-07,8996.940430,8996.940430,8719.919922,8769.700195,8769.700195,266710000
4,2009-01-08,8769.940430,8770.019531,8651.190430,8742.459961,8742.459961,226620000


In [8]:
seq_length = 30
step_length = 5

In [9]:
df['target_log'] = np.log(df['Close'].shift(-step_length) / df['Close'])

In [130]:
train_df = df[:int(length*0.6)].copy()
train_df.reset_index(inplace=True)
val_df = df[int(length*0.6):int(length*0.8)].copy()
val_df.reset_index(inplace=True)
test_df = df.loc[int(length*0.8):].copy()
test_df.reset_index(inplace=True)
train_df.shape, val_df.shape, test_df.shape

((1660, 9), (553, 9), (554, 9))

In [103]:
def create_candlebar(df, setname):
    target = []
    filename = []
    if not os.path.exists(root_dir + 'data/charts/' + setname):
        os.makedirs(root_dir + 'data/charts/' + setname)
    for i in range(df.shape[0] - step_length - seq_length):
        print(i+1, end='\r')
        target.append(df.loc[i+seq_length-1, 'target_log'])
        slices = df[i: i+seq_length]
        plt.figure(figsize=(1.12, 1.12), dpi=100)
        candlestick2_ohlc(plt.subplot(211), slices['Open'].values, slices['High'].values,
                        slices['Low'].values, slices['Close'].values, width=0.6,
                        colorup='b', colordown='r', alpha=1)
        plt.axis('off')

        plt.subplot(212)
        plt.bar(range(30), slices['Volume'].values, color='g')
        plt.axis('off')

        plt.savefig(root_dir + 'data/charts/' + setname + '/candlebar_'\
                    + str(i) + '-' + str(i+seq_length-1) + '.png')
        filename.append('candlebar_' + str(i) + '-' + str(i+seq_length-1) + '.png')
        plt.close('all')
    pd.DataFrame({'filename': filename,
                  'target': target}).to_csv(root_dir + 'data/charts/' + setname + '_target.csv', index=False)

In [104]:
create_candlebar(train_df, 'train')

In [105]:
create_candlebar(val_df, 'val')